In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.35.2 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install -

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import pandas as pd
import os
import torch
import numpy as np
from scipy import stats
# Path to your env.txt file
env_file_path = 'data/env.txt'

# Read and set environment variables
with open(env_file_path, 'r') as file:
    for line in file:
        key, value = line.strip().split('=')
        os.environ[key] = value
token = os.environ['huggingface_token']

## Check Length of MATHWELL Training Data and Prompts

In [4]:
df = pd.read_csv('data/mathwell_annotations_final.csv')
model_path = "meta-llama/Llama-2-70b-hf"   # Specify the path to the model
tokenizer = AutoTokenizer.from_pretrained(model_path, token=token)

question_lengths = 0
for i in range(0, len(df)):
    output = df.iloc[i]['output']
    try: 
        inputs = tokenizer.encode(output, return_tensors="pt")
    except:
        pass
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average question length in tokens: {question_lengths/len(df)}")

In [5]:
df = pd.read_csv('data/mathwell.csv')
import random
topics = ['Superman', "Batman", "Wonder Woman", "Barbie", "Power Rangers", "basketball", "soccer", "football", "volleyball", 'field hockey',\
'Fortnite', 'Spiderman', "Iron Man", "Captain America", "Captain Marvel", "Thor, the God of Thunder", "Ninja Turtles", "Black Panther", "Taylor Swift", "swimming",\
"Pokémon", "Super Mario", "Naruto", "unicorns", "Hello Kitty", "Minecraft", "lacrosse", "cheer leading", "LeBron James", "Steph Curry", "Patrick Mahomes",\
"Serena Williams", "dogs", "cats", "dinosaurs", "Harry Potter", "cars", "planes", "trains", "pizza", "cookies", "ice cream", 'candy']
question_lengths = 0
for i in range(0,1000):
    topic = random.choice(topics)
    final_prompt = f"Write a grade school math word problem about {topic} and Python function with a commented out step-by-step solution to solve the word problem."
    prompt = "Write a grade school math word problem and Python function with a commented out step-by-step solution to solve the word problem."
    questions = []
    for i in range(0, 8):
        question = df['output'].iloc[random.randint(0,len(df)-1)]
        questions.append(question)
    formatted_prompt = []
    for i in range(0,8):
        formatted_prompt.append((f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{prompt}\n\n### Response: {questions[i]}"))
    formatted_prompt.append(f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{final_prompt}\n\n### Response: ")
    formatted_prompt = "\n".join(formatted_prompt)
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average input length in tokens: {question_lengths/1000}")

Average input length in tokens: 2052.738


In [5]:
def check_length(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['text']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_mathwell(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_length_good(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths.append(length)
    return question_lengths
def check_gsm8k(df):
    question_lengths = []
    for i in range(0, len(df)):
        output = str(df.iloc[i]['instruction'])
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        if length>2:
            question_lengths.append(length)
    return question_lengths

## Check Average Length of GSM8K Training Questions

In [9]:
gsm8k = pd.read_csv('data/gsm8k_questions.csv')
gsm8k = gsm8k.drop_duplicates(subset = 'instruction')
gsm8k = check_gsm8k(gsm8k)
gsm8k = np.array(gsm8k)
np.save('gsm8k_len.npy', gsm8k)
print(f'Average overall question length in tokens: {np.mean(gsm8k)} Standard Dev: {np.std(gsm8k)}')

Average overall question length in tokens: 66.23613930969857 Standard Dev: 23.928523886898294


## Check Average Length of SGSM Unannotated

In [10]:
sgsm_unan = pd.read_csv('data/sgsm_unannotated.csv')
sgsm_unan_len = check_mathwell(sgsm_unan)
sgsm_unan_len = np.array(sgsm_unan_len)
np.save('sgsm_unan_len.npy', sgsm_unan_len)
print(f'Average overall question length in tokens: {np.mean(sgsm_unan_len)} Standard Dev: {np.std(sgsm_unan_len)}')

Average overall question length in tokens: 62.51584497472414 Standard Dev: 14.775757907904286


## Check Length of MATHWELL Training Data, Questions Only

In [13]:
mathwell_an = pd.read_csv('data/mathwell_annotations_final.csv')
mathwell_an_len = check_mathwell(mathwell_an)
print(f'Average overall question length in tokens: {np.mean(mathwell_an_len)} Standard Dev: {np.std(mathwell_an_len)}')

Average overall question length in tokens: 58.90872560275545 Standard Dev: 16.540447022330724


In [14]:
mathwell_train = mathwell_an[mathwell_an['good']==1]
mathwell_train_len = check_length_good(mathwell_train)
sgsm_train_len = np.array(mathwell_train_len)
np.save('sgsm_train_len.npy', sgsm_train_len)
print(f'Average good question length in tokens: {np.mean(mathwell_train_len)} Standard Dev: {np.std(mathwell_train_len)}')

Average good question length in tokens: 57.22360248447205 Standard Dev: 15.697440213554898


## Check Average Length of Overall SGSM Dataset

In [16]:
mathwell_train = mathwell_an[mathwell_an['good']==1]
sgsm = pd.concat([sgsm_unan, mathwell_train])
sgsm_len = check_mathwell(sgsm)
sgsm_len = np.array(sgsm_len)
np.save('sgsm_len.npy', sgsm_len)
print(f'Average overall question length in tokens: {np.mean(sgsm_len)} Standard Dev: {np.std(sgsm_len)}')

Average overall question length in tokens: 61.975256222547586 Standard Dev: 14.95863248895829


## Check Length of MATHWELL Final Questions

In [17]:
mathwell_all = pd.read_csv('data/mathwell_solvability.csv')
mathwell = check_length(mathwell_all)
mathwell_len = np.array(mathwell)
np.save('mathwell_len.npy', mathwell_len)
print(f'Average overall question length in tokens: {np.mean(mathwell)} Standard Error: {stats.sem(mathwell)}')

Average overall question length in tokens: 54.656 Standard Error: 0.8550010310240619


In [18]:
mathwell_good = pd.read_csv('data/mathwell_good.csv')
mathwell_good = check_length_good(mathwell_good)
mathwell_mac_len = np.array(mathwell_good)
np.save('mathwell_mac_len.npy', mathwell_mac_len)
print(f'Average good question length in tokens: {np.mean(mathwell_good)} Standard Error: {stats.sem(mathwell_good)}')

Average good question length in tokens: 54.12299465240642 Standard Error: 0.9739900006033011


## Check Length of Llama Questions

In [19]:
llama_all = pd.read_csv('data/llama_solvability.csv')
llama = check_length(llama_all)
llama_len = np.array(llama)
np.save('llama_len.npy', llama_len)
print(f'Average overall question length in tokens: {np.mean(llama)} Standard Error: {stats.sem(llama)}')

Average overall question length in tokens: 51.62 Standard Error: 0.9936901329495454


In [20]:
llama_good = pd.read_csv('data/llama_good.csv')
llama_good = check_length_good(llama_good)
llama_mac_len = np.array(llama_good)
np.save('llama_mac_len.npy', llama_mac_len)
print(f'Average good question length in tokens: {np.mean(llama_good)} Standard Error: {stats.sem(llama_good)}')

Average good question length in tokens: 49.81410256410256 Standard Error: 1.187152221057565


## Check Length of LLEMA Questions

In [21]:
llema_all = pd.read_csv('data/llema_solvability.csv')
llema = check_length(llema_all)
llema_len = np.array(llema)
np.save('llema_len.npy', llema_len)
print(f'Average overall question length in tokens: {np.mean(llema)} Standard Error: {stats.sem(llema)}')

Average overall question length in tokens: 56.376 Standard Error: 1.4372254394684996


In [22]:
llema_good = pd.read_csv('data/llema_good.csv')
llema_good = check_length_good(llema_good)
llema_mac_len = np.array(llema_good)
np.save('llema_mac_len.npy', llema_mac_len)
print(f'Average good question length in tokens: {np.mean(llema_good)} Standard Error: {stats.sem(llema_good)}')

Average good question length in tokens: 50.921052631578945 Standard Error: 2.8867632326739767


## Check Length of MAmmoTH Questions

In [23]:
mammoth_all = pd.read_csv('data/mammoth_solvability.csv')
mammoth = check_length(mammoth_all)
mammoth_len = np.array(mammoth)
np.save('mammoth_len.npy', mammoth_len)
print(f'Average overall question length in tokens: {np.mean(mammoth)} Standard Error: {stats.sem(mammoth)}')

Average overall question length in tokens: 45.876 Standard Error: 1.133237576866603


In [24]:
mammoth_good = pd.read_csv('data/mammoth_good.csv')
mammoth_good = check_length_good(mammoth_good)
mammoth_mac_len = np.array(mammoth_good)
np.save('mammoth_mac_len.npy', mammoth_mac_len)
print(f'Average good question length in tokens: {np.mean(mammoth_good)} Standard Error: {stats.sem(mammoth_good)}')

Average good question length in tokens: 44.36619718309859 Standard Error: 1.1546779975033012


## Check Length of NumGLUE

In [26]:
numglue = pd.read_csv('data/numglue_questions.csv')
numglue = numglue.drop_duplicates(subset = 'instruction')
numglue_len = check_gsm8k(numglue)
numglue_len = np.array(numglue_len)
np.save('numglue_len.npy', numglue_len)
print(f'Average overall question length in tokens: {np.mean(numglue_len)} Standard Dev: {np.std(numglue_len)}')

Average overall question length in tokens: 144.82678816224498 Standard Dev: 136.48286031167407


## Check Length of ASDIV

In [21]:
asdiv = pd.read_csv('data/ASDiv_clean.csv')
asdiv_len = check_length_good(asdiv)
asdiv_len = np.array(asdiv_len)
np.save('asdiv_len.npy', asdiv_len)
print(f'Average overall question length in tokens: {np.mean(asdiv_len)} Standard Error: {stats.sem(asdiv_len)}')

Average overall question length in tokens: 45.06898047722343 Standard Error: 0.32820245347269494


## Check Length of SVAMP

In [22]:
svamp = pd.read_json('data/SVAMP.json')
svamp['question'] = svamp['Body'] + " " + svamp['Question']
svamp_len = check_length_good(svamp)
svamp_len = np.array(svamp_len)
np.save('svamp_len.npy', svamp_len)
print(f'Average overall question length in tokens: {np.mean(svamp_len)} Standard Error: {stats.sem(svamp_len)}')

Average overall question length in tokens: 47.268 Standard Error: 0.37014210944065956


## Check Length of GSM-Hard

In [23]:
gsm_hard = pd.read_json('data/gsmhard.json')
gsm_hard_len = check_length_good(gsm_hard)
gsm_hard_len = np.array(gsm_hard_len)
np.save('gsm_hard_len.npy', gsm_hard_len)
print(f'Average overall question length in tokens: {np.mean(gsm_hard_len)} Standard Error: {stats.sem(gsm_hard_len)}')

Average overall question length in tokens: 72.9294920394238 Standard Error: 0.7061476070833672


## Tests for Statistical Significance

In [24]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(numglue_len, gsm_hard_len)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 19.325426920836847
P-value: 3.938957671833916e-82


In [27]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llema, mathwell)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 1.0285132122923446
P-value: 0.30420775782105663


In [28]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(gsm8k, mathwell_train_len)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 16.131843568188767
P-value: 1.0912118926155136e-57


In [29]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(gsm8k, sgsm_unan_len)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 14.566949893696288
P-value: 7.188494718675924e-48


In [30]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llema, mathwell)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: 1.0285132122923446
P-value: 0.30420775782105663


In [31]:
# Perform an independent two-sample t-test on solvability
t_statistic, p_value = stats.ttest_ind(llema_good, mathwell_good)

# Print the results
print("T-statistic:", t_statistic)
print("P-value:", p_value)

T-statistic: -1.2707808220078132
P-value: 0.20513070636311811
